In [1]:
import illustris_python as il
import requests
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import os

/home/xingyzt/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
baseUrl = 'http://www.tng-project.org/api/TNG-Cluster'
headers = {"api-key":os.getenv('API_KEY')}
base_dir = 'tng_cache/'

def get(path, params=None, save_dir=''):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = base_dir + save_dir + r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [4]:
sim = get(baseUrl)
sim

{'name': 'TNG-Cluster',
 'description': 'Main high-resolution TNG-Cluster simulation suite (virtual box), with the full fiducial TNG physics model.',
 'name_alt': 'L680n8192TNG',
 'boxsize': 680000.0,
 'z_start': 127.0,
 'z_final': 0.0,
 'cosmology': 'Planck2015',
 'omega_0': 0.3089,
 'omega_L': 0.6911,
 'omega_B': 0.0486,
 'hubble': 0.6774,
 'physics_model': 'Fiducial TNG Model',
 'has_cooling': True,
 'has_starformation': True,
 'has_winds': True,
 'has_blackholes': True,
 'mass_gas': 0.000771495,
 'mass_dm': 0.00413211373831018,
 'softening_dm_comoving': 2.0,
 'softening_stars_comoving': 2.0,
 'softening_blackholes_comoving': 4.0,
 'softening_gas_comoving': 2.0,
 'softening_dm_max_phys': 1.0,
 'softening_stars_max_phys': 1.0,
 'softening_blackholes_max_phys': 4.0,
 'softening_gas_max_phys': 1.0,
 'softening_gas_factor': 2.5,
 'softening_gas_comoving_min': 0.25,
 'num_dm': 549755813888,
 'num_tr_mc': 15416383432,
 'num_tr_vel': 0,
 'longids': False,
 'is_uniform': True,
 'is_zoom': F

In [5]:
# get z = 0 snapshot
snap_index = 99
snap_dir = f'snap_{snap_index:03d}/'
snap_dir

'snap_099/'

In [7]:
snaps = get(sim['snapshots'])
snap = get(snaps[snap_index]['url'], save_dir=snap_dir)
snap

{'simulation': 'http://www.tng-project.org/api/TNG-Cluster/',
 'number': 99,
 'redshift': 0.0,
 'num_gas': 14969664891,
 'num_dm': 14829431232,
 'num_trmc': 15416383432,
 'num_stars': 1067549783,
 'num_bhs': 430359,
 'num_groups_fof': 19453039,
 'num_groups_subfind': 10378451,
 'num_groups_rockstar': 0,
 'filesize_snapshot': 4928426793596.0,
 'filesize_groupcat': 16110992699.0,
 'filesize_rockstar': 0.0,
 'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/',
 'subhalos': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/',
 'vis': {'groupcat_sfms': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=ssfr&xlim=9.0%2C11.0',
  'groupcat_sizemass': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=size_stars',
  'groupcat_mz_gas': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/plot.png?xQuant=mstar2_log&yQuant=Z_gas',
  'groupcat_mz_stars': 'http

In [9]:
subs = get( snap['subhalos'], {'limit':50, 'order_by':'-mass_stars'})
subs

{'count': 10378451,
 'next': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/?limit=50&offset=50&order_by=-mass_stars',
 'previous': None,
 'results': [{'id': 4680430,
   'sfr': 6.98751,
   'mass_log_msun': 14.823941436818016,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/4680430/'},
  {'id': 2930523,
   'sfr': 0.100453,
   'mass_log_msun': 15.20857685911439,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/2930523/'},
  {'id': 845400,
   'sfr': 0.0373764,
   'mass_log_msun': 15.405578694454283,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/845400/'},
  {'id': 8143526,
   'sfr': 320.685,
   'mass_log_msun': 14.61170512428078,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/8143526/'},
  {'id': 6419490,
   'sfr': 1924.89,
   'mass_log_msun': 14.651030081787413,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/6419490/'},
  {'id':

In [25]:
sub_index = 20
# get (sub_index)th most massive subhalo and its siblings
sub = get( subs['results'][sub_index]['url'] )
sub

{'snap': 99,
 'id': 1087173,
 'bhmdot': 0.00107797,
 'cm_x': 179754.0,
 'cm_y': 456810.0,
 'cm_z': 103474.0,
 'gasmetallicity': 0.00352256,
 'gasmetallicityhalfrad': 0.00372972,
 'gasmetallicitymaxrad': 0.00293808,
 'gasmetallicitysfr': 0.0,
 'gasmetallicitysfrweighted': 0.0,
 'pos_x': 179790.0,
 'pos_y': 456638.0,
 'pos_z': 103491.0,
 'halfmassrad': 962.826,
 'halfmassrad_gas': 956.026,
 'halfmassrad_dm': 970.692,
 'halfmassrad_stars': 94.4079,
 'halfmassrad_bhs': 0.0,
 'len': 47921712,
 'len_gas': 20658438,
 'len_dm': 25910878,
 'len_stars': 1352195,
 'len_bhs': 201,
 'mass': 125069.0,
 'mass_gas': 17267.8,
 'mass_dm': 107067.0,
 'mass_stars': 729.947,
 'mass_bhs': 4.61596,
 'massinhalfrad': 2578.73,
 'massinhalfrad_gas': 76.2591,
 'massinhalfrad_dm': 2133.07,
 'massinhalfrad_stars': 364.973,
 'massinhalfrad_bhs': 4.435,
 'massinmaxrad': 57321.0,
 'massinmaxrad_gas': 7859.71,
 'massinmaxrad_dm': 48796.7,
 'massinmaxrad_stars': 660.024,
 'massinmaxrad_bhs': 4.55692,
 'massinrad': 6909

In [29]:
parent_request = { 'limit': '100' }
parent = get(sub['related']['parent_halo'], params=parent_request )
parent_info = get(parent['meta']['info'])
parent_info

{'SimulationName': 'L680n8192TNG',
 'SnapshotNumber': 99,
 'InfoType': 'Group',
 'InfoID': 2069291,
 'GroupBHMass': 5.610498428344727,
 'GroupBHMdot': 0.021090680733323097,
 'GroupCM': [179215.5, 457670.0625, 103050.1796875],
 'GroupContaminationFracByMass': 0.0,
 'GroupContaminationFracByNumPart': 0.0,
 'GroupFirstSub': 1087173,
 'GroupGasMetalFractions': [0.7551436424255371,
  0.24254167079925537,
  0.00024397083325311542,
  7.937035843497142e-05,
  0.0011704128701239824,
  0.0003801144193857908,
  0.00011171689402544871,
  0.00010579774243524298,
  0.00014786979591008276,
  7.540979277109727e-05],
 'GroupGasMetallicity': 0.002314662793651223,
 'GroupLen': 70956668,
 'GroupLenType': [27905504, 38658279, 0, 0, 4392384, 501],
 'GroupMass': 185396.484375,
 'GroupMassType': [23292.712890625,
  159740.40625,
  0.0,
  0.0,
  2351.909912109375,
  11.451870918273926],
 'GroupNsubs': 10433,
 'GroupOffsetType': [654684199, 1034940314, 4220810, 0, 113454690, 42985],
 'GroupOrigHaloID': 11,
 'Gr

In [30]:
parent['child_subhalos']

{'count': 10433,
 'next': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/halos/2069291/?limit=100&offset=100',
 'previous': None,
 'results': [{'id': 1087173,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087173/'},
  {'id': 1087174,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087174/'},
  {'id': 1087175,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087175/'},
  {'id': 1087176,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087176/'},
  {'id': 1087177,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087177/'},
  {'id': 1087178,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087178/'},
  {'id': 1087179,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087179/'},
  {'id': 1087180,
   'url': 'http://www.tng-project.org/api/TNG-Cluster/snapshots/99/subhalos/1087180/'},
 

In [31]:
sibling_subs = [ 
    get(res['url']) for res in parent['child_subhalos']['results']
]
# array entries
sub_keys = set(sibling_subs[0].keys()) - { 'related', 'cutouts', 'trees', 'supplementary_data', 'vis', 'meta' }
sub_keys


{'bhmdot',
 'cm_x',
 'cm_y',
 'cm_z',
 'desc_sfid',
 'desc_snap',
 'gasmetallicity',
 'gasmetallicityhalfrad',
 'gasmetallicitymaxrad',
 'gasmetallicitysfr',
 'gasmetallicitysfrweighted',
 'grnr',
 'halfmassrad',
 'halfmassrad_bhs',
 'halfmassrad_dm',
 'halfmassrad_gas',
 'halfmassrad_stars',
 'id',
 'len',
 'len_bhs',
 'len_dm',
 'len_gas',
 'len_stars',
 'mass',
 'mass_bhs',
 'mass_dm',
 'mass_gas',
 'mass_log_msun',
 'mass_stars',
 'massinhalfrad',
 'massinhalfrad_bhs',
 'massinhalfrad_dm',
 'massinhalfrad_gas',
 'massinhalfrad_stars',
 'massinmaxrad',
 'massinmaxrad_bhs',
 'massinmaxrad_dm',
 'massinmaxrad_gas',
 'massinmaxrad_stars',
 'massinrad',
 'massinrad_bhs',
 'massinrad_dm',
 'massinrad_gas',
 'massinrad_stars',
 'parent',
 'pos_x',
 'pos_y',
 'pos_z',
 'primary_flag',
 'prog_sfid',
 'prog_snap',
 'sfr',
 'sfrinhalfrad',
 'sfrinmaxrad',
 'sfrinrad',
 'snap',
 'spin_x',
 'spin_y',
 'spin_z',
 'starmetallicity',
 'starmetallicityhalfrad',
 'starmetallicitymaxrad',
 'stellarph

In [32]:
parent_request = {
    'gas':'Coordinates,Masses,ParticleIDs,Density,ElectronAbundance,GFM_Metals',
    'stars':'Coordinates,Masses,ParticleIDs,GFM_StellarFormationTime'
}
parent_cutout = get(sub['cutouts']['parent_halo'], parent_request, save_dir=snap_dir)


In [33]:
sibling_request = {
    'gas':'ParticleIDs',
    'stars':'ParticleIDs'
}
for sub in sibling_subs:
    if sub['len_gas'] + sub['len_stars'] > 0:
        get(sub['cutouts']['subhalo'], sibling_request, save_dir=snap_dir)

In [34]:
with h5py.File(base_dir + snap_dir + f'cutout_{parent_info["InfoID"]}.hdf5', 'r') as f:
    gas_ids = f['PartType0/ParticleIDs'][:]
    star_ids = f['PartType4/ParticleIDs'][:]

    gas_sub_ids = -np.ones_like(gas_ids, dtype=np.int64)
    star_sub_ids = -np.ones_like(star_ids, dtype=np.int64)

for sub in sibling_subs:
    if sub['len_gas'] + sub['len_stars'] > 0:
        with h5py.File(base_dir + snap_dir + f'cutout_{sub["id"]}.hdf5', 'r') as sub_f:
            
            if sub['len_gas'] > 0:
                sub_gas_ids = sub_f['PartType0/ParticleIDs'][:]
                gas_sub_ids[np.isin(gas_ids, sub_gas_ids)] = sub["id"]
            
            if sub['len_stars'] > 0:
                sub_star_ids = sub_f['PartType4/ParticleIDs'][:]
                star_sub_ids[np.isin(star_ids, sub_star_ids)] = sub["id"]

with h5py.File(base_dir + snap_dir + f'cutout_{parent_info["InfoID"]}.hdf5', 'a') as f:
        f['PartType0/SubhaloIDs'] = gas_sub_ids
        f['PartType4/SubhaloIDs'] = star_sub_ids

In [35]:
with h5py.File("outputs/dm_proto.hdf5", "a") as f:
    g_halo = f.create_group(f'halo_{parent_info["InfoID"]}')
    g_halo['GroupCM'] = parent_info['GroupCM']
    g_halo['GroupMassType'] = parent_info['GroupMassType']
    g_halo['GroupSFR'] = parent_info['GroupSFR']
    g_halo['Group_R_Crit500'] = parent_info['Group_R_Crit500']
    g_subs = g_halo.create_group('subhalos')
    for k in sub_keys:
        g_subs[k] = [ s[k] for s in sibling_subs ]

In [36]:
with h5py.File("outputs/dm_proto.hdf5", "r") as f:
    print(f.keys())

<KeysViewHDF5 ['halo_12925051', 'halo_15386300', 'halo_2069291']>
